### 1 根据离散概率分布采样

In [5]:
import random
 # input: probability distribution and correspondence
list_probability = [0.005, 0.015, 0.08, 0.25, 0.3, 0.25, 0.08, 0.015, 0.005]
list_player_role = ['黑寡妇', '蜘蛛侠', '绿巨人', '雷神', '钢铁侠', '奇异博士', '美国队长', '黑豹', '鹰眼']
# sampling
result = random.choices(list_player_role, weights=list_probability, k=1)
# output: sampling one by probability distribution
print(result)


['雷神']


In [2]:
import random
import numpy as np
p = [0.005, 0.015, 0.08, 0.25, 0.3, 0.25, 0.08, 0.015, 0.005]
index = np.arange(len(p))
probability_index = random.choices(index, weights=list_probability, k=4)
print(probability_index)

NameError: name 'list_probability' is not defined

#### 线性抽样概率设计

In [34]:
num_sample = 500
batch_size = 128
p = np.arange(num_sample,0,-1)
p = p/sum(p)
index = np.random.choice(np.arange(num_sample), replace=False, size=batch_size, p=p)

In [8]:
one_step_error = [0.5]*100
one_step_error.pop()
one_step_error.insert(0, 0.1)

In [10]:
len(one_step_error)

100

### 网格搜索超参数

In [5]:
class Hyperparams:
    def __init__(self, param:{}):
        self.param = param
        self.param_num = len(self.param)
        self.param_keys = list(self.param.keys())
        self.param_comb = []
        self.get_param()
        
    def get_param(self,param_dict={}, current_num=0):
        if current_num==self.param_num:
            self.param_comb.append(param_dict)
        else:
            for param in self.param[self.param_keys[current_num]]:
                param_dict.update({self.param_keys[current_num]:param})
                self.get_param(param_dict=param_dict.copy(), current_num=current_num+1)

### 参数不同模型不同，遍历参数组合，重构模型
### 保留每个epoch验证集和测试集的损失，
### 记录每个模型在150个epoch下，验证集loss最小值
param = {'hidden_size':[16, 32, 64],'layers':[2, 3, 4],'weight_decay':[1e-1,1e-2,1e-3,1e-4],'dropout':[0.1,0.2,0.4,0.6]}
hyperparams = Hyperparams(param)
# grid_search
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print('training on', device)
# loss = nn.L1Loss(reduction ='sum')
# epochs = 150
# val_loss_min = np.zeros((len(hyperparams.param_comb), 5))
# train_loss_min = np.zeros((len(hyperparams.param_comb), 5))
# for i,p in enumerate(hyperparams.param_comb):
#     # 重建网络
#     print(p)
#     net = RNNModel(feature_size=16, hidden_size=p['hidden_size'], num_layers=p['layers'], dropout=p['dropout'])
#     net.to(dtype=torch.float32, device=device)
#     net.apply(init_net)
#     optimizer = torch.optim.Adam(net.parameters(), lr=0.001, weight_decay=p['weight_decay'])
#     # 每个参数下训练5次
#     for j in range(5):
#         val_loss_min[i,j], train_loss_min[i,j] = train_net(net, train_dl, val_dl, optimizer, loss, device, epochs)
#     print(val_loss_min[i,:], train_loss_min[i,:])
#     break

In [10]:
import random
import numpy as np
# 超参数优化结果
val_loss_min = np.load(r'result\val_loss_min.npy')
train_loss_min = np.load(r'result\train_loss_min.npy')

In [11]:
val_loss_min_mean = val_loss_min.mean(1)
val_loss_min_std = val_loss_min.std(1)
train_loss_min_mean = train_loss_min.mean(1)
train_loss_min_std = train_loss_min.std(1)
for i in val_loss_min_mean.argsort()[0:7]:
    print(hyperparams.param_comb[i])
    print('val_mean',val_loss_min_mean[i])
    print('val_std',val_loss_min_std[i])
    print('train_mean',train_loss_min_mean[i])
    print('train_std',train_loss_min_std[i])

{'hidden_size': 64, 'layers': 2, 'weight_decay': 0.001, 'dropout': 0.1}
val_mean 1.8475780181884764
val_std 0.7284600360033135
train_mean 1.2901618954755258
train_std 0.1709701254689933
{'hidden_size': 64, 'layers': 2, 'weight_decay': 0.001, 'dropout': 0.2}
val_mean 2.0550709974500867
val_std 0.44853302974009246
train_mean 1.8151175628883252
train_std 0.41698171972144915
{'hidden_size': 32, 'layers': 2, 'weight_decay': 0.01, 'dropout': 0.1}
val_mean 2.123810840182834
val_std 0.43803324409582656
train_mean 1.7163538764179616
train_std 0.3523229663614422
{'hidden_size': 32, 'layers': 2, 'weight_decay': 0.001, 'dropout': 0.1}
val_mean 2.2155883009168837
val_std 0.5816161944579709
train_mean 1.8081633330082547
train_std 0.37824978574575824
{'hidden_size': 64, 'layers': 2, 'weight_decay': 0.001, 'dropout': 0.4}
val_mean 2.263553675333659
val_std 0.21309375907809852
train_mean 2.041412507209225
train_std 0.18176179739358814
{'hidden_size': 32, 'layers': 3, 'weight_decay': 0.0001, 'dropout': 

### MAS源代码解析

In [ ]:
# if continual_learning and loss_window_mean < self.loss_window_mean_threshold and loss_window_variance < self.loss_window_variance_threshold and new_peak_detected:
if continual_learning and new_peak_detected:
    count_updates+=1
    update_tags.append(0.01)
    last_loss_window_mean=loss_window_mean
    last_loss_window_variance=loss_window_variance
    new_peak_detected=False
    # calculate importance weights and update star_variables
    gradients=[0 for p in self.model.parameters()]
    # calculate imporatance based on each sample in the hardbuffer
    for sx in [_['state'] for _ in hard_buffer]:
        self.model.zero_grad()
        y_pred=self.model(torch.from_numpy(np.asarray(sx).reshape(-1,self.dim)).type(torch.float32))
        torch.norm(y_pred, 2, dim=1).backward()
        for pindex, p in enumerate(self.model.parameters()):
            g=p.grad.data.clone().detach().numpy()
            gradients[pindex]+=np.abs(g)
            
            
# 训练过程
for gs in range(self.gradient_steps):
    # evaluate the new batch
    y_pred = self.model(torch.from_numpy(np.asarray(x).reshape(-1,self.dim)).type(torch.float32))
    y_sup=torch.zeros(len(y),2).scatter_(1,torch.from_numpy(np.asarray(y).reshape(-1,1)).type(torch.LongTensor),1.).type(torch.FloatTensor)
    recent_loss = self.loss_fn(y_pred,y_sup)
    total_loss = torch.sum(self.loss_fn(y_pred,y_sup))

    if use_hard_buffer and len(hard_buffer) != 0:
        # evaluate hard buffer
        yh_pred = self.model(torch.from_numpy(np.asarray(xh).reshape(-1,self.dim)).type(torch.float32))
        yh_sup=torch.zeros(len(yh),2).scatter_(1,torch.from_numpy(np.asarray(yh).reshape(-1,1)).type(torch.LongTensor),1.).type(torch.FloatTensor)

        hard_loss = self.loss_fn(yh_pred,yh_sup)
        total_loss += torch.sum(self.loss_fn(yh_pred,yh_sup))

    # keep train loss for loss window
    if gs==0: first_train_loss=total_loss.detach().numpy()

    # add MAS regularization to the training objective
    if continual_learning and len(star_variables)!=0 and len(omegas)!=0:
        for pindex, p in enumerate(self.model.parameters()):
            total_loss+=self.MAS_weight/2.*torch.sum(torch.from_numpy(omegas[pindex]).type(torch.float32)*(p-star_variables[pindex])**2)

    # train self.model
    self.optimizer.zero_grad()
    torch.sum(total_loss).backward()
    self.optimizer.step()

    # save training accuracy on total batch
    if use_hard_buffer and len(hard_buffer) != 0:
    xt=x+xh
    yt=y+yh
    else:
    xt=x[:]
    yt=y[:]
    yt_pred = self.model(torch.from_numpy(np.asarray(xt).reshape(-1,self.dim)).type(torch.float32))
    accuracy = np.mean(np.argmax(yt_pred.detach().numpy(),axis=1)==yt)
    msg+=' recent loss: {0:0.3f}'.format(np.mean(recent_loss.detach().numpy()))
    if use_hard_buffer and len(hard_buffer) != 0:
    msg+=' hard loss: {0:0.3f}'.format(np.mean(hard_loss.detach().numpy()))
    losses.append(np.mean(accuracy))

In [ ]:
# 计算重要性权重
gradients=[0 for p in net.parameters()]   # 初始化，用来保存单个样本在线更新时的梯度
if continual_learning and error<thd:  # 仅在误差较小的情况下保持网络稳定性权值
    # 计算各个参数的重要性权重
    # 分析重要性权重 并做分布调整 如果有需要的话
    # calculate imporatance based on each sample
    # 统计更新次数
    count_updates += 1 
    # 梯度清零
    net.zero_grad()
    # 获得预测结果,因为是在线的方式这是单个样本
    torch.norm(y_pred, 2, dim=1).backward() # 多步的稳定性
    y_pred=net(X);  # 这是多步预测结果呀
    # 梯度backward()
    y_pred.backward()
    # 保存grad，对梯度做分布map处理
    for pindex, p in enumerate(net.parameters()):
        g=p.grad.data.clone().detach().numpy()
        gradients[pindex]+=np.abs(g)

# update the running average of the importance weights        
# 文章中采用的平均权重，这里采用时间衰减权重
    omegas_old = omegas[:]  # 上次的omegas，内存地址没变
    omegas=[]  # 当前的omegas
    star_variables=[]
    for pindex, p in enumerate(net.parameters()):
        if len(omegas_old) != 0:   # 不是初始化omegas
            omegas.append(1/count_updates*gradients[pindex]+(1-1/count_updates)*omegas_old[pindex])
        else:
            omegas.append(gradients[pindex])
        star_variables.append(p.data.clone().detach())  # 保存此时的参数，用来构造Loss


# 计算损失，训练模型，源代码中是最近buffer满了之后再训练
# Train model on replaybuffer when it is full:
for epoch in range(epochs):
    # evaluate the new batch
    y_pred = net(x)
    recent_loss = loss(y_pred,y)
    total_loss = torch.sum(loss(y_pred,y_sup))  # 这里求和
    
    # evaluate hard buffer
    yh_pred = net(xh)
    hard_loss = loss(yh_pred,yh)
    total_loss += torch.sum(loss_fn(yh_pred,yh_sup))
    
    # add MAS regularization to the training objective
    if continual_learning:
        for pindex, p in enumerate(net.parameters()):
            total_loss += l*torch.sum(omegas[pindex]*(p-star_variables[pindex])**2)
# train self.model
optimizer.zero_grad()
torch.sum(total_loss).backward()
optimizer.step()

### 转换操作
.detach().numpy()\
torch.from_numpy()

In [ ]:
### 绘图
config = {
"font.family":'serif',  
"font.size": 14,       #14 放大是16
"mathtext.fontset":'stix',
"xtick.direction":"in",
"ytick.direction":"in",
#"font.serif": ['SimSun'],
"font.family":"times new roman",
}
rcParams.update(config)
color_list = plt.cm.tab10(np.linspace(0, 1, 12))
x = np.arange(1,16)
fig, ax1 = plt.subplots(2,2, figsize=(5,4), dpi=160)

points = [16000,18000,23000,25000]
point = points[0]
ax1[0,0].plot(x, y_true[point-1,:], color=color_list[4], linestyle='-', linewidth=2)
ax1[0,0].plot(x, y_pred[point-1,:], color=color_list[0], linestyle='-.', linewidth=2)
ax1[0,0].set_title('No.10')


point = points[1]
ax1[0,1].plot(x, y_true[point-1,:], color=color_list[4], linestyle='-', linewidth=2)
ax1[0,1].plot(x, y_pred[point-1,:], color=color_list[0], linestyle='-.', linewidth=2)
ax1[0,1].set_title('No.460')

point = points[2]
ax1[1,0].plot(x, y_true[point-1,:], color=color_list[4], linestyle='-', linewidth=2)
ax1[1,0].plot(x, y_pred[point-1,:], color=color_list[0], linestyle='-.', linewidth=2)

ax1[1,0].set_title('No.990')

point = points[3]   # 1380
ax1[1,1].plot(x, y_true[point-1,:], color=color_list[4], linestyle='-', linewidth=2)
ax1[1,1].plot(x, y_pred[point-1,:], color=color_list[0], linestyle='-.', linewidth=2)

ax1[1,1].set_title('No.1580')
#ax1[1,1].set_ylim([290,320])

egend_font = {"family" : "serif"}
fig.legend(['TRUE', 'LSTM', ],frameon=False,\
           fontsize='x-small', bbox_to_anchor=(1.18, 0.92))

fig.text(0.5, 0, "Horizon", ha='center')
fig.text(0, 0.5, "NOx($\mathrm{mg/m^{3}}$)", va='center', rotation='vertical')   
plt.tight_layout()